![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`

| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`

| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


**1. Find the 3 best-performing industries based on the number of new unicorns created over three years (2019, 2020, and 2021) combined.**

In [84]:
SELECT 
    i.industry,
    COUNT(i.*) AS count_new_unicorns
FROM industries i
JOIN dates d ON i.company_id = d.company_id
WHERE DATE_PART('year', d.date_joined) IN ('2019', '2020', '2021')
GROUP BY i.industry
ORDER BY count_new_unicorns DESC
LIMIT 3

,industry,count_new_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


**2. Calculate the number of unicorns and the average valuation, grouped by year and industry.**

In [85]:
SELECT i.industry,
		DATE_PART('year', d.date_joined) AS year,
    	COUNT(i.*) AS num_unicorns,
    	AVG(f.valuation) AS average_valuation
FROM industries i
JOIN dates d ON i.company_id = d.company_id
JOIN funding f ON d.company_id = f.company_id
GROUP BY i.industry, year

,industry,year,num_unicorns,average_valuation
0,Mobile & telecommunications,2017,5,2.800000e+09
1,Internet software & services,2015,4,1.250000e+09
2,Fintech,2018,10,8.600000e+09
3,Mobile & telecommunications,2019,4,2.000000e+09
4,Artificial intelligence,2012,1,2.000000e+09
...,...,...,...,...
119,E-commerce & direct-to-consumer,2022,7,1.571429e+09
120,Other,2015,2,1.000000e+09
121,Consumer & retail,2017,4,1.050000e+10
122,Fintech,2019,20,6.800000e+09


**3. Create 2 CTEs for 2 two tables above and run**

In [86]:
WITH top_industries AS (
	SELECT 
    	i.industry, 
        COUNT(i.*)
    FROM industries AS i
    JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE DATE_PART('year', d.date_joined) IN ('2019', '2020', '2021')
    GROUP BY industry
    ORDER BY count DESC
    LIMIT 3
),

yearly_ranks AS 
(
    SELECT 
    	COUNT(i.*) AS num_unicorns,
        i.industry,
        DATE_PART('year', d.date_joined) AS year,
        AVG(f.valuation) AS average_valuation
    FROM industries AS i
    JOIN dates AS d
        ON i.company_id = d.company_id
    JOIN funding AS f
        ON d.company_id = f.company_id
    GROUP BY industry, year
)

SELECT 
	industry,
    year,
    num_unicorns,
    ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
FROM yearly_ranks
WHERE year in ('2019', '2020', '2021')
	AND industry in (SELECT industry FROM top_industries)
GROUP BY industry, num_unicorns, year, average_valuation
ORDER BY industry, year DESC

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
